In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/business.csv
/kaggle/input/economy.csv
/kaggle/input/Clean_Dataset.csv


In [3]:
# Load dataset into DataFrame and identify columns for missing rows and consistent data types
df = pd.read_csv('/kaggle/input/Clean_Dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        300153 non-null  int64  
 1   airline           300153 non-null  object 
 2   flight            300153 non-null  object 
 3   source_city       300153 non-null  object 
 4   departure_time    300153 non-null  object 
 5   stops             300153 non-null  object 
 6   arrival_time      300153 non-null  object 
 7   destination_city  300153 non-null  object 
 8   class             300153 non-null  object 
 9   duration          300153 non-null  float64
 10  days_left         300153 non-null  int64  
 11  price             300153 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 27.5+ MB


In [4]:
# Inspect the outputs of the DataFrame
df.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [5]:
# Check the value counts in percentages for any inconsistent outputs and data clearning
columns = df.drop(['Unnamed: 0', 'price'], axis=1).columns

for index, column in enumerate(columns):
    print(f'{df[column].value_counts(normalize=True)*100}\n')

airline
Vistara      42.597942
Air_India    26.950255
Indigo       14.366007
GO_FIRST      7.720396
AirAsia       5.363265
SpiceJet      3.002136
Name: proportion, dtype: float64

flight
UK-706     1.077784
UK-772     0.913201
UK-720     0.882883
UK-836     0.846901
UK-822     0.822247
             ...   
SG-9923    0.000333
6E-6474    0.000333
6E-865     0.000333
G8-705     0.000333
G8-107     0.000333
Name: proportion, Length: 1561, dtype: float64

source_city
Delhi        20.437244
Mumbai       20.288320
Bangalore    17.344821
Kolkata      15.441125
Hyderabad    13.595067
Chennai      12.893424
Name: proportion, dtype: float64

departure_time
Morning          23.703245
Early_Morning    22.251985
Evening          21.689605
Night            15.996842
Afternoon        15.923212
Late_Night        0.435111
Name: proportion, dtype: float64

stops
one            83.578375
zero           11.995216
two_or_more     4.426409
Name: proportion, dtype: float64

arrival_time
Night            30.49

In [6]:
# Drop unneeded columns or features and separate define the target
X = df.drop(['Unnamed: 0', 'flight', 'price'], axis=1)
y = df['price']

X.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1


In [7]:
# Encode all categorical/object variables into dummy variables
from sklearn.preprocessing import StandardScaler, OneHotEncoder 

# Identify the categorical variables in churn_df
X_cat_columns = X.select_dtypes(include=['object']).columns.tolist()

# Convert categorical features in churn_df into features_scaled
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat_encoded = enc.fit_transform(X[X_cat_columns])
X_cat_encoded_df = pd.DataFrame(X_cat_encoded, columns=enc.get_feature_names_out(X_cat_columns))

# Select remaining numerical variables in churn_df
X_num_columns = X.select_dtypes(include=['int64','float64']).columns.tolist()
X_num_df = X[X_num_columns]

# Merge categorical and numerical variables in churn_df before scaling
features_df = X_num_df.merge(X_cat_encoded_df, left_index=True, right_index=True)

features_df.head()

,duration,days_left,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Bangalore,source_city_Chennai,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,2.17,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2.33,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2.17,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,2.25,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,2.33,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [8]:
# Perform feature scaling scale them
scr = StandardScaler()

# Define your scaled features and target variable for the churn prediction model
features_scaled = scr.fit_transform(features_df)

print(features_scaled)

[[-1.39753079 -1.84387477 -0.23805908 ...  2.01965066 -0.67257558
   0.67257558]
 [-1.3752838  -1.84387477 -0.23805908 ...  2.01965066 -0.67257558
   0.67257558]
 [-1.39753079 -1.84387477  4.20063801 ...  2.01965066 -0.67257558
   0.67257558]
 ...
 [ 0.22371837  1.69569214 -0.23805908 ... -0.49513513  1.48682175
  -1.48682175]
 [-0.30881888  1.69569214 -0.23805908 ... -0.49513513  1.48682175
  -1.48682175]
 [-0.29769538  1.69569214 -0.23805908 ... -0.49513513  1.48682175
  -1.48682175]]


In [9]:
# Create train and test data splits
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_scaled, y, test_size=0.3, random_state=123)

In [37]:
# Import both individual algorithms and ensemble methods
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score, mean_squared_error

In [12]:
# Prepare and create voting regressor ensemble model
voting_estimators = [
    ('rid', Ridge()),
    ('knr', KNeighborsRegressor(n_neighbors=3)),
    ('rfr', RandomForestRegressor(n_estimators=25))]

voting_reg = VotingRegressor(estimators=voting_estimators)

voting_reg.fit(X_train, y_train)

y_pred_voting_reg = voting_reg.predict(X_test)

In [13]:
# Prepare and create bagging regressor ensemble model
bagging_estimator = DecisionTreeRegressor(max_depth=3)
                               
bagging_reg = BaggingRegressor(estimator=bagging_estimator, n_estimators=10, bootstrap=True, random_state=123)

bagging_reg.fit(X_train, y_train)

y_pred_bagging_reg = bagging_reg.predict(X_test)

In [16]:
# Prepare and create stacking regressor ensemble model
stacking_estimators = [
    ('rid', Ridge()),
    ('knr', KNeighborsRegressor(n_neighbors=3)),
    ('dtr', DecisionTreeRegressor(max_depth=3))]

final_estimator = RandomForestRegressor(random_state=123)

stacking_reg = StackingRegressor(estimators=stacking_estimators, final_estimator=final_estimator)

stacking_reg.fit(X_train, y_train)

y_pred_stacking_reg = stacking_reg.predict(X_test)

In [17]:
# Prepare and create xgboost regressor ensemble model
xgboosting_reg = xgb.XGBRegressor(random_state=123)

xgboosting_reg.fit(X_train, y_train)

y_pred_xgboosting_reg = xgboosting_reg.predict(X_test)

In [23]:
# Calculate the accuracy score  or coefficient of determination (R2) for each ensemble model
voting_score = r2_score(y_test, y_pred_voting_reg)

bagging_score = r2_score(y_test, y_pred_bagging_reg)

stacking_score = r2_score(y_test, y_pred_stacking_reg)

xgboosting_score = r2_score(y_test, y_pred_xgboosting_reg)

print("Voting Score: ", voting_score)
print("Bagging Score: ", bagging_score)
print("Stacking Score: ", stacking_score)
print("XGBoosting Score: ", xgboosting_score)

Voting Score:  0.9745284226685919
Bagging Score:  0.9350426868109007
Stacking Score:  0.969788961350351
XGBoosting Score:  0.9771571953012846


In [38]:
# Conduct Hyperparameter Tuning for the best performing ensemble model so far to see if score can improve
from sklearn.model_selection import RandomizedSearchCV

xgb_param_grid = {
    'booster': ['gblinear', 'gbtree'],
    'learning_rate': np.arange(0.05, 1.0, 0.05),
    'subsample': np.arange(0.3, 1.0, 0.1)
}

randomized_xgb = RandomizedSearchCV(estimator=xgboosting_reg, param_distributions=xgb_param_grid, scoring='r2', n_iter=20, cv=4, verbose=1, random_state=123)

randomized_xgb.fit(X_train, y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=123, ...),
                   n_iter=20,
                   param_distributions={'booster': ['gblinear', 'gbtree'],
                                        'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
                                        'subsample': array([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
                   random_state=123, scoring='r2', verbose=1)

In [41]:
print("RandomizedCV of XGBoosting Best Params: ", randomized_xgb.best_params_)
print("\nRandomizedCV of XGBoosting Best Score: ", randomized_xgb.best_score_)

RandomizedCV of XGBoosting Best Params:  {'subsample': 0.9000000000000001, 'learning_rate': 0.6500000000000001, 'booster': 'gbtree'}

RandomizedCV of XGBoosting Best Score:  0.9803143880017682
